# Initial Setup Code

In [ ]:
# get everything set up
# more rapidly install node
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# install other dependencies
!pip install transformers
!pip install datasets
# install repo with the data
!git clone https://github.com/BerenMillidge/svd_directions
%cd svd_directions

!bash setup.sh

import torch
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from copy import deepcopy
from tqdm.auto import tqdm, trange
import re
from collections import defaultdict
from transformers import AutoModelForCausalLM, AutoTokenizer
# utils
import json
from torch import nn
import torch.nn.functional as F
from datasets import load_dataset
from copy import deepcopy
from torch.nn import functional as F
from tabulate import tabulate
from tqdm import tqdm, trange
import functools
import math

# this resets up the site so you don't have to restart the runtime to use pysvelte
import site
site.main()
# import pysvelte


# sns.set_palette('colorblind')
# cmap = sns.color_palette('colorblind')


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

                           SCRIPT DEPRECATION WARNING                    

  
  This script, located at https://deb.nodesource.com/setup_X, used to
  install Node.js is deprecated now and will eventually be made inactive.

  Please visit the NodeSource distributions Github and follow the
  instructions to migrate your repo.
  https://github.com/nodesource/distributions

  The NodeSource Node.js Linux distributions GitHub repository contains
  information about which versions of Node.js and which Linux distributions
  are supported and how to install it.
  https://github.com/nodesource/distributions


                          SCRIPT DEPRECATION WARNING

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

TO AVOID THIS WAIT MIGRATE THE SCRIPT
Continuing in 60 seconds (press Ctrl-C to abort) ...


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache.

In [ ]:
def keep_k(x, k=100, absolute=True, dim=-1):
    shape = x.shape
    x_ = x
    if absolute:
        x_ = abs(x)
    values, indices = torch.topk(x_, k=k, dim=dim)
    res = torch.zeros_like(x)
    res.scatter_(dim, indices, x.gather(dim, indices))
    return res

def get_max_token_length(tokens):
  maxlen = 0
  for t in tokens:
    l = len(t)
    if l > maxlen:
      maxlen = l
  return maxlen

def pad_with_space(t, maxlen):
  spaces_to_add = maxlen - len(t)
  for i in range(spaces_to_add):
    t += " "
  return t

def convert_to_tokens(indices, tokenizer, extended, extra_values_pos, strip=True, pad_to_maxlen=False):
    if extended:
        res = [tokenizer.convert_ids_to_tokens([idx])[0] if idx < len(tokenizer) else
               (f"[pos{idx-len(tokenizer)}]" if idx < extra_values_pos else f"[val{idx-extra_values_pos}]")
               for idx in indices]
    else:
        res = tokenizer.convert_ids_to_tokens(indices)
    if strip:
        res = list(map(lambda x: x[1:] if x[0] == 'Ġ' else "#" + x, res))
    if pad_to_maxlen:
      maxlen = get_max_token_length(res)
      res = list(map(lambda t: pad_with_space(t, maxlen), res))
    return res


def top_tokens(v_tok, k=100, tokenizer=None, only_english=False, only_ascii=True, with_values=False,
               exclude_brackets=False, extended=True, extra_values=None, pad_to_maxlen=False):
    if tokenizer is None:
        tokenizer = my_tokenizer
    v_tok = deepcopy(v_tok)
    ignored_indices = []
    if only_ascii:
        ignored_indices = [key for val, key in tokenizer.vocab.items() if not val.strip('Ġ').isascii()]
    if only_english:
        ignored_indices =[key for val, key in tokenizer.vocab.items() if not (val.strip('Ġ').isascii() and val.strip('Ġ[]').isalnum())]
    if exclude_brackets:
        ignored_indices = set(ignored_indices).intersection(
            {key for val, key in tokenizer.vocab.items() if not (val.isascii() and val.isalnum())})
        ignored_indices = list(ignored_indices)
    v_tok[ignored_indices] = -np.inf
    extra_values_pos = len(v_tok)
    if extra_values is not None:
        v_tok = torch.cat([v_tok, extra_values])
    values, indices = torch.topk(v_tok, k=k)
    res = convert_to_tokens(indices, tokenizer, extended=extended, extra_values_pos=extra_values_pos,pad_to_maxlen = pad_to_maxlen)
    if with_values:
        res = list(zip(res, values.cpu().numpy()))
    return res


def top_matrix_tokens(mat, k=100, tokenizer=None, rel_thresh=None, thresh=None,
                      sample_entries=10000, alphabetical=True, only_english=False,
                      exclude_brackets=False, with_values=True, extended=True):
    if tokenizer is None:
        tokenizer = my_tokenizer
    mat = deepcopy(mat)
    ignored_indices = []
    if only_english:
        ignored_indices = [key for val, key in tokenizer.vocab.items() if not (val.isascii() and val.strip('[]').isalnum())]
    if exclude_brackets:
        ignored_indices = set(ignored_indices).intersection(
            {key for val, key in tokenizer.vocab.items() if not (val.isascii() and val.isalnum())})
        ignored_indices = list(ignored_indices)
    mat[ignored_indices, :] = -np.inf
    mat[:, ignored_indices] = -np.inf
    cond = torch.ones_like(mat).bool()
    if rel_thresh:
        cond &= (mat > torch.max(mat) * rel_thresh)
    if thresh:
        cond &= (mat > thresh)
    entries = torch.nonzero(cond)
    if sample_entries:
        entries = entries[np.random.randint(len(torch.nonzero(cond)), size=sample_entries)]
    res_indices = sorted(entries,
                         key=lambda x: x[0] if alphabetical else -mat[x[0], x[1]])
    res = [*map(partial(convert_to_tokens, extended=extended, tokenizer=tokenizer), res_indices)]

    if with_values:
        res_ = []
        for (x1, x2), (i1, i2) in zip(res, res_indices):
            res_.append((x1, x2, mat[i1][i2].item()))
        res = res_
    return res

In [ ]:
def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

def get_model_tokenizer_embedding(model_name="gpt2-medium"):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  if device == 'cpu':
    print("WARNING: you should probably restart on a GPU runtime")

  model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  emb = model.get_output_embeddings().weight.data.T.detach()
  return model, tokenizer, emb, device


def get_model_info(model):
  num_layers = model.config.n_layer
  num_heads = model.config.n_head
  hidden_dim = model.config.n_embd
  head_size = hidden_dim // num_heads
  return num_layers, num_heads, hidden_dim, head_size

def get_mlp_weights(model,num_layers, hidden_dim):
  Ks = []
  Vs = []
  for j in range(num_layers):
    K = model.get_parameter(f"transformer.h.{j}.mlp.c_fc.weight").T.detach()
    # fuse the layernorm
    ln_2_weight = model.get_parameter(f"transformer.h.{j}.ln_2.weight").detach()
    K = torch.einsum("oi,i -> oi", K, ln_2_weight)

    V = model.get_parameter(f"transformer.h.{j}.mlp.c_proj.weight")
    Ks.append(K)
    Vs.append(V)

  Ks =  torch.cat(Ks)
  Vs = torch.cat(Vs)
  K_heads = Ks.reshape(num_layers, -1, hidden_dim)
  V_heads = Vs.reshape(num_layers, -1, hidden_dim)
  return K_heads, V_heads

def get_attention_heads(model, num_layers, hidden_dim, num_heads, head_size):
  qkvs = []
  for j in range(num_layers):
    qkv = model.get_parameter(f"transformer.h.{j}.attn.c_attn.weight").detach().T
    ln_weight_1 = model.get_parameter(f"transformer.h.{j}.ln_1.weight").detach()

    qkv = qkv - torch.mean(qkv, dim=0)
    qkv = torch.einsum("oi,i -> oi", qkv, ln_weight_1)
    qkvs.append(qkv.T)

  W_Q, W_K, W_V = torch.cat(qkvs).chunk(3, dim=-1)
  W_O = torch.cat([model.get_parameter(f"transformer.h.{j}.attn.c_proj.weight") for j in range(num_layers)]).detach()
  W_V_heads = W_V.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
  W_O_heads = W_O.reshape(num_layers, num_heads, head_size, hidden_dim)
  W_Q_heads = W_Q.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
  W_K_heads = W_K.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
  return W_Q_heads, W_K_heads, W_V_heads, W_O_heads

def top_singular_vectors(mat, emb, all_tokens, k = 20, N_singular_vectors = 10, with_negative = False,use_visualization=True, filter="topk"):
  U,S,V = torch.linalg.svd(mat)
  Vs = []
  for i in range(N_singular_vectors):
      acts = V[i,:].float() @ emb
      Vs.append(acts)
  if use_visualization:
    Vs = torch.stack(Vs, dim=1).unsqueeze(1) # n_tokens, n_layers (1), n_directions
    pysvelte.TopKTable(tokens=all_tokens, activations=Vs, obj_type="SVD direction", k=k, filter=filter).show()
  else:
    Vs = [top_tokens(Vs[i].float().cpu(), k = k, pad_to_maxlen=True) for i in range(len(Vs))]
    print(tabulate([*zip(*Vs)]))
  if with_negative:
    Vs = []
    for i in range(N_singular_vectors):
      acts = -V[i,:].float() @ emb
      Vs.append(acts)
    if use_visualization:
      Vs = torch.stack(Vs, dim=1).unsqueeze(1) # n_tokens, n_layers (1), n_directions
      pysvelte.TopKTable(tokens=all_tokens, activations=Vs, obj_type="SVD direction", k=k, filter=filter).show()
    else:
      Vs = [top_tokens(Vs[i].float().cpu(), k = k, pad_to_maxlen=True) for i in range(len(Vs))]
      print(tabulate([*zip(*Vs)]))

def plot_MLP_singular_vectors(K,layer_idx, max_rank=None):
  W_matrix = K[layer_idx, :,:]
  U,S,V = torch.linalg.svd(W_matrix,full_matrices=False)
  if not max_rank:
    max_rank = len(S)
  if max_rank > len(S):
    max_rank = len(S) -1
  plt.plot(S[0:max_rank].detach().cpu().numpy())
  plt.yscale('log')
  plt.ylabel("Singular value")
  plt.xlabel("Rank")
  plt.title("Distribution of the singular vectors")
  plt.show()

def cosine_sim(x,y):
    return torch.dot(x,y) / (torch.norm(x) * torch.norm(y))


def normalize_and_entropy(V, eps=1e-6):
    absV = torch.abs(V)
    normV = absV / torch.sum(absV)
    entropy = torch.sum(normV * torch.log(normV + eps)).item()
    return -entropy


## Load model and get weights

In [ ]:
# Load up the model and get all the key weight matrices.
model, tokenizer, emb, device = get_model_tokenizer_embedding()
my_tokenizer = tokenizer
num_layers, num_heads, hidden_dim, head_size = get_model_info(model)
all_tokens = [tokenizer.decode([i]) for i in range(tokenizer.vocab_size)]

K,V = get_mlp_weights(model, num_layers = num_layers, hidden_dim = hidden_dim)
W_Q_heads, W_K_heads, W_V_heads, W_O_heads = get_attention_heads(model, num_layers=num_layers, hidden_dim=hidden_dim, num_heads=num_heads, head_size = head_size)


# GPT-4 Autolabelling Code

In [ ]:
%%capture
!pip install openai
import openai, os
openai.api_key = ""  # provide API key here

## Find singular vectors representing Agents

In [ ]:
# some utility function
import json
def remove_hashes(l):
    new_l = []
    for el in l:
        s = el
        s = s.replace("#","")
        new_l.append(s)
    return new_l

def list_to_comma_separated_string(l):
    s = ""
    for el in l:
        s+= str(el) + ", "
    return s[:-2]

# def get_mlp_in_V(model):
def get_mlp_in_V(j):
    # W = model.get_parameter(f"transformer.h.{j}.mlp.c_fc.weight").T.detach()
    # U,S,V = torch.linalg.svd(W)
    W_matrix = K[j, :,:]
    # W_matrix = K[layer_idx, :,:]
    U,S,V = torch.linalg.svd(W_matrix,full_matrices=False)
    return V

# def get_mlp_out_V(model):
def get_mlp_out_V(j):
    W = model.get_parameter(f"transformer.h.{j}.mlp.c_proj.weight").T.detach()
    U,S,V = torch.linalg.svd(W)
    return V

def get_model_responses(model_name, mlp_type, mlp_layer, direction, save_base = "data/json_data/"):
  # load json file and parse
#   model_type = model_name.split("-")[1]
#   with open(save_base + model_type + "_" + mlp_type + "_text_outputs.json",'r') as json_data:
  model_type = model_name
  with open(save_base + model_type + "_" + mlp_type + "_text_outputs.json",'r') as json_data:
      json_str = json_data.read()
      json_dict = json.loads(json_str)
      json_data.close()
  result_dict = json_dict["layer_" + str(mlp_layer)]["singdir_" + str(direction)]
  return result_dict

def print_result(result_dict):
  print("SVD direction: " + str(result_dict["test_str"]))
  print("Model response: " + str(result_dict["out_text"]))
#   print("Verifier response: " + str(result_dict["verifier_response"]))

Run the following prompt through GPT-4 first to see what is obtained:

In [ ]:
meaningless_str = "house door balloon wind red the going eating blue sky lamp shade food ape doctor"
animal_str = "cat house dog door puppy wind human red puppies eating the going eating blue sky"
names_string = "Emily Bob David John Eve Sophia house door balloon wind red the going eating blue"

openai_prompt = "Words that are agents include: people, names, occupations (doctor, etc), roles (friend, relative, etc), and animals. Agents must be both nouns and living beings. Do not count uncommon names and last names as agents."
openai_prompt += "If at least 30% of the words in the input after the question 'Are at least 30% of these words agents?' are agents, answer with: \"Yes\".  \n"
openai_prompt += "Then state which words you believe are agents: \n"
openai_prompt += "Else answer with \" No.\".  \n"
openai_prompt += "Here are a few examples: \n"

openai_prompt += "Are at least 30% of these words agents? \n"
openai_prompt += meaningless_str + " \n"
openai_prompt += "Answer: No. \n"

openai_prompt += "Are at least 30% of these words agents? \n"
openai_prompt += animal_str + " \n"
openai_prompt += "Answer: Yes. \n"
openai_prompt += "cat dog puppy human puppies \n"

openai_prompt += "Are at least 30% of these words agents? \n"
openai_prompt += names_string + " \n"
openai_prompt += "Answer: Yes. \n"
openai_prompt += "Emily Bob David John Eve Sophia \n"

openai_prompt += "Are at least 30% of these words agents? \n"
test_str = "Bunny, Turtle, Way, Penny, damn, ONEY, Goose, Plenty, Hope, Cowboy, enne, King, town, Happy, damned"
openai_prompt += test_str + " \n"
openai_prompt += "Answer:"
openai_prompt

'Words that are agents include: people, names, occupations (doctor, etc), roles (friend, relative, etc), and animals. Agents must be both nouns and living beings. Do not count uncommon names and last names as agents.If at least 30% of the words in the input after the question \'Are at least 30% of these words agents?\' are agents, answer with: "Yes".  \nThen state which words you believe are agents: \nElse answer with " No.".  \nHere are a few examples: \nAre at least 30% of these words agents? \nhouse door balloon wind red the going eating blue sky lamp shade food ape doctor \nAnswer: No. \nAre at least 30% of these words agents? \ncat house dog door puppy wind human red puppies eating the going eating blue sky \nAnswer: Yes. \ncat dog puppy human puppies \nAre at least 30% of these words agents? \nEmily Bob David John Eve Sophia house door balloon wind red the going eating blue \nAnswer: Yes. \nEmily Bob David John Eve Sophia \nAre at least 30% of these words agents? \nBunny, Turtle,

In [ ]:
def check_agents(test_str):
    meaningless_str = "house door balloon wind red the going eating blue sky lamp shade food ape doctor"
    animal_str = "cat house dog door puppy wind human red puppies eating the going eating blue sky"
    names_string = "Emily Bob David John Eve Sophia house door balloon wind red the going eating blue"

    openai_prompt = "Words that are agents include: people, names, occupations (doctor, etc), roles (friend, relative, etc), and animals. Agents must be both nouns and living beings. Do not count uncommon names and last names as agents."
    openai_prompt += "If at least 30% of the words in the input after the question 'Are at least 30% of these words agents?' are agents, answer with: \"Yes\".  \n"
    openai_prompt += "Then state which words you believe are agents: \n"
    openai_prompt += "Else answer with \" No.\".  \n"
    openai_prompt += "Here are a few examples: \n"

    openai_prompt += "Are at least 30% of these words agents? \n"
    openai_prompt += meaningless_str + " \n"
    openai_prompt += "Answer: No. \n"

    openai_prompt += "Are at least 30% of these words agents? \n"
    openai_prompt += animal_str + " \n"
    openai_prompt += "Answer: Yes. "
    openai_prompt += "cat dog puppy human puppies \n"

    openai_prompt += "Are at least 30% of these words agents? \n"
    openai_prompt += names_string + " \n"
    openai_prompt += "Answer: Yes. "
    openai_prompt += "Emily Bob David John Eve Sophia \n"

    openai_prompt += "Are at least 30% of these words agents? \n"
    openai_prompt += test_str + " "
    openai_prompt += "Answer:"
    print("test str: ", test_str)

    openai_output = openai.ChatCompletion.create(
        model = 'gpt-4', # model_name
        messages = [{"role": "user", "content": openai_prompt}],
    )
    out_text = openai_output.choices[0]["message"]["content"].strip()
    print("output: ", out_text)
    if "Yes." in out_text or 'yes.' in out_text:
        return 1, openai_prompt, out_text
    else:
        return 0, openai_prompt, out_text

In [ ]:
val, prompt, out_text = check_agents("Bunny, Turtle, Way, Penny, damn, ONEY, Goose, Plenty, Hope, Cowboy, enne, King, town, Happy, damned")

test str:  Bunny, Turtle, Way, Penny, damn, ONEY, Goose, Plenty, Hope, Cowboy, enne, King, town, Happy, damned
output:  Yes. Bunny, Turtle, Goose, Cowboy, King


In [ ]:
val, prompt, out_text = check_agents("Mub Hayden Notting Mau Tig Nebula Lindsey Bron Baghd Torment Guerrero mun Wil Kurd Yel")

test str:  Mub Hayden Notting Mau Tig Nebula Lindsey Bron Baghd Torment Guerrero mun Wil Kurd Yel
output:  No.


The singular vectors, their top-k tokens, and their auto-labeled results were already saved in the 'data' folder in the original authors' repo, so just load them (no need to auto-label again).

In [ ]:
def sweep_mlp_layers(model,emb, model_name, N_singular_vectors=3, mlp_fn = get_mlp_in_V, save_base = "new_data/", prompt_function = check_agents):
    model_str = model_name.replace("-","_")
    num_layers, num_heads, hidden_dim, head_size = get_model_info(model)
    # num_layers = 1
    corrects_list = [] # list of corrects arrays
    text_dict = {}  # layer : layer_dict ; use to query by [layer] to get layer_dict, then query by [sing_vec_ind]
    for n in range(0, num_layers):
        corrects = []  # ints of 1 or 0. 1 means can interpret

        # V = mlp_fn(n) # get_mlp_in_V(n) obtains the singular vectors

        layer_dict = {}  # sing_vec_ind : sing_dict ; loop over top 10 sing vec w/ highest sing values
        for i in range(N_singular_vectors):
            sing_dict = {}  # input:text, output:text

            # test_str =  list_to_comma_separated_string(remove_hashes(top_tokens(V.T[i,:].float() @ emb, k = k)))
            with open("/content/svd_directions/data/json_data/medium_mlp_in_text_outputs.json", "r") as file:
                data = json.load(file)
            interpretation = data['layer_'+str(n)]['direction_'+str(i)]['response']
            if "no shared semantic meaning" not in interpretation:
                test_str = data['layer_'+str(n)]['direction_'+str(i)]['test_string'] # load top tokens for (layer, singvec) from json
                test_str = test_str.replace(",", "")

                print("MLP Layer: ", n, "; Singular Vec: ", i)
                print(data['layer_'+str(n)]['direction_'+str(i)]['response'])
                val, prompt, out_text = prompt_function(test_str)
                corrects.append(val)
                sing_dict["test_str"] = test_str
                sing_dict["out_text"] = out_text
                sing_dict["interpret"] = interpretation
                layer_dict["singdir_" + str(i)] = sing_dict

        text_dict["layer_" + str(n)] = layer_dict
        corrects_list.append(np.array(corrects))
        np.save(save_base + model_str + "_mlp_in_corrects_list.npy", corrects_list)
        json_str = json.dumps(text_dict)
        json_sname = save_base + model_str + "_mlp_in_text_outputs.json"
        with open(json_sname, "w") as f:
            f.write(json_str)
            f.close()
    corrects_list = np.array(corrects_list)
    np.save(save_base + model_str + "_mlp_in_corrects_list.npy", corrects_list)  # save corrects_list in data folder
    json_str = json.dumps(text_dict)
    json_sname = save_base + model_str + "_mlp_in_text_outputs.json" # save text outputs
    with open(json_sname, "w") as f:
        f.write(json_str)
        f.close()
    return corrects_list, text_dict

In [ ]:
os.makedirs("/content/svd_directions/agents_data")

In [ ]:
mlp_in_corrects_list, mlp_in_text_dict = sweep_mlp_layers(model, emb, model_name="gpt2_medium", N_singular_vectors=30, mlp_fn = get_mlp_in_V, save_base = "agents_data/")

MLP Layer:  0 ; Singular Vec:  0
most of these words are computer related terms.
test str:  Canaver umbn ItemTracker Interstitial TPPStreamerBot ngth kef Seym SourceFile DragonMagazine irlf izoph \\\\\\\\ BuyableInstoreAndOnline destro
output:  No.
MLP Layer:  0 ; Singular Vec:  1
most of these words relate to computer science.
test str:  hem oks NetMessage pload asive gered wiser Pastebin ventus venge alm escription eering ignt Ire
output:  No.
MLP Layer:  0 ; Singular Vec:  4
most of these words are verbs.
test str:  depended without relied iences seeks also ecause relies actually ienced thus ogether nonetheless consists served
output:  No.
MLP Layer:  0 ; Singular Vec:  5
most of these words are place names.
test str:  Mub Hayden Notting Mau Tig Nebula Lindsey Bron Baghd Torment Guerrero mun Wil Kurd Yel
output:  Yes. Mub Hayden Notting Mau Tig Lindsey Bron Kurd Yel
MLP Layer:  0 ; Singular Vec:  6
most of these words are adjectives.
test str:  understandable tesy unintentionally in

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


output:  No.
MLP Layer:  3 ; Singular Vec:  3
most of these words relate to government.
test str:  national constituted comprising Government constit Legislative settlement Cause Amendments Jur Federal constitutes National jointly Legisl
output:  No.
MLP Layer:  3 ; Singular Vec:  4
most of these words are places.
test str:  BG Lara Arena Academy Garcia Champions Center revamped Burg Costa Stadium Mar AK Lake Camp
output:  No.
MLP Layer:  3 ; Singular Vec:  6
most of these words relate to religion
test str:  Juda sama Commands speech Han aloud addons hier Accept discrim arta venge interrog decrypt ahu
output:  No.
MLP Layer:  3 ; Singular Vec:  7
most of these words are related to the business world
test str:  trending locally federally actively globally pricing backlog Trend Developers Trend financially sourcing customizable operators nationally
output:  No.
MLP Layer:  3 ; Singular Vec:  8
most of these words are related to female gender.
test str:  emate owship backdoor enny igious 

<ipython-input-88-73a1f35fb248>:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  corrects_list = np.array(corrects_list)


Because we are located within the temporary closed repo (see 'pwd'), the new_data folder will be made within the temporary cloned repo. Download its data by:

In [ ]:
!zip -r labeled_dirs.zip /content/svd_directions/agents_data/
from google.colab import files
files.download('labeled_dirs.zip')

  adding: content/svd_directions/agents_data/ (stored 0%)
  adding: content/svd_directions/agents_data/gpt2_medium_mlp_in_text_outputs.json (deflated 71%)
  adding: content/svd_directions/agents_data/gpt2_medium_mlp_in_corrects_list.npy (deflated 86%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now we can query these saved outputs by (layer, direction). We can also print out all the results that were identified to contain agent concepts.

In [ ]:
# print saved text outputs for a certain (mlp, sing_vec)
responses = get_model_responses(model_name="gpt2_medium", mlp_type="mlp_in", mlp_layer=0, direction=0, save_base = "agents_data/")
print_result(responses)

SVD direction: Canaver umbn ItemTracker Interstitial TPPStreamerBot ngth kef Seym SourceFile DragonMagazine irlf izoph \\\\\\\\ BuyableInstoreAndOnline destro
Model response: no.


In [ ]:
with open("/content/svd_directions/agents_data/gpt2_medium_mlp_in_text_outputs.json", "r") as file:
    data = json.load(file)

# Iterate over each layer and its sub-entries to print entries containing "yes" in "out_text"
for layer, sub_entries in data.items():
    for singdir, values in sub_entries.items():
        if 'yes' in values['out_text'].lower():
            test_string = values['test_str']
            out_text = values['out_text']
            out_text = out_text.replace("yes.", "")
            interpretation = values['interpret']
            print(layer, singdir)
            print(f"{test_string}")
            print(f"Agent words: {out_text}")
            print(f"{interpretation}\n")

layer_0 singdir_5
Mub Hayden Notting Mau Tig Nebula Lindsey Bron Baghd Torment Guerrero mun Wil Kurd Yel
Agent words: Yes. Mub Hayden Notting Mau Tig Lindsey Bron Kurd Yel
most of these words are place names.

layer_0 singdir_18
Bunny Turtle Way Penny damn ONEY Goose Plenty Hope Cowboy enne King town Happy damned
Agent words: Yes. Bunny Turtle Penny Goose Hope Cowboy King
most of these words are nouns.

layer_1 singdir_22
troopers trooper usc beam locks itiz lock strom killed buster arresting murderer caches sheriff Jae
Agent words: Yes. troopers trooper buster murderer sheriff Jae
most of these words relate to law enforcement.

layer_2 singdir_16
Either Attempt trib enemy anged ERROR forfeit guessed sei decimal nder ursed error opponent penalties
Agent words: Yes. Attempt trib enemy opponent
most of these words are related to trying or failing

layer_3 singdir_9
philosopher writer builder creator strategist Publisher theorist agenda Founding urer jugg philosophers suprem Wars publishe

Let's find the number of and fraction of interpretable directions corresponding to agent concepts:

In [ ]:
with open("/content/svd_directions/agents_data/gpt2_medium_mlp_in_text_outputs.json", "r") as file:
    data = json.load(file)

# Counter for results containing 'yes' in 'out_text'
count_yes = 0
total = 0

# Iterate over each layer and its sub-entries to count entries containing "yes" in "out_text"
for layer, sub_entries in data.items():
    for singval, values in sub_entries.items():
        if 'yes' in values['out_text'].lower():
            count_yes += 1
        total += 1

print(count_yes)
print(count_yes/total)

78
0.1262135922330097
